## This script finds the mean distance from a resident of an informal or atomistic area to the nearest open space.

### Method:

1. Reclass ULU: [2, 3] --> [1, 1], and 0 otherwise. This gives 1 for informal/atomistic. Call this **informal**.
1. Multiply WorldPop raster by **informal**. This gives popsize only in informal pixels, zero otherwise. Call this **informal_pop**.
1. Create distance raster giving distance within 2000m of OSM openspace features. Call this **openspace_distance**.
1. Multiply **informal_pop** by **openspace_distance**. Call this **informal_persondistancetoopenspace**.
1. Sum over **informal_pop** to get total number of residents of informal/atomistic. Call this **total_informal_pop**.
1. Return **informal_persondistancetoopenspace** / **total_informal_pop**. This is the average distance from an informal/atomistic dwelling to openspace.


In [1]:
import os, requests, json
from math import floor
import pyproj
import overpy
import pandas as pd
import geemap
import ee
import shapely
from shapely.geometry import Polygon, MultiPolygon
from shapely.geometry import shape as Shape
from shapely.ops import unary_union, transform
#ee.Authenticate()

Enter verification code: 4/1AZEOvhXZ4XsYe2RFIU_tMaEIPCv8rZuXzNZV9DggvDF8ryI-7fhjkKKEgZs

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [29]:
if False:
    # define directory
    out_dir = os.getcwd()
    bucket_name = 'cities-indicators'
    aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com"

    # get list of c4f cities
    boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
    
    geos = []
    for i in range(len(boundary_georef)):
        boundary_id_aoi = '{0}-{1}'.format(boundary_georef.loc[i, 'geo_name'], boundary_georef.loc[i, 'aoi_boundary_name'])
        boundary_path = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-{0}.geojson'.format(aws_s3_dir, boundary_id_aoi)
        boundary_geo = requests.get(boundary_path).json()
        geos.append((boundary_georef.loc[i, 'country_code'], boundary_georef.loc[i, 'city_name'], boundary_georef.loc[i, 'geo_level'], boundary_geo))

In [36]:
boundaries_foldername = 'Deep Dive boundaries'
geos = []
for city_geojson_filename in os.listdir(boundaries_foldername):
    info_part = city_geojson_filename.split('.')[0]
    bfile = open('{0}/{1}'.format(boundaries_foldername, city_geojson_filename), encoding="utf-8")
    country_code = info_part.split('-')[1]
    cityname = info_part.split('-')[2]
    level = '-'.join(info_part.split('-')[3:])
    geos.append((country_code, cityname, level, bfile.read()))
    bfile.close()

#Jakarta
boundary_georef = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
i = 25
boundary_id_aoi = '{0}-{1}'.format(boundary_georef.loc[i, 'geo_name'], boundary_georef.loc[i, 'aoi_boundary_name'])
boundary_path = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-{0}.geojson'.format(boundary_id_aoi)
boundary_geo = requests.get(boundary_path).json()
geos.append((boundary_georef.loc[i, 'country_code'], boundary_georef.loc[i, 'city_name'], boundary_georef.loc[i, 'units_boundary_name'], json.dumps(boundary_geo)))

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


In [8]:
POP_YEAR = 2020
OPENSPACE_YEAR = 2022

In [9]:
# Get ULU polygons

ULU200 = ee.ImageCollection('projects/wri-datalab/urban_land_use/V1')
ULU4000 = ee.ImageCollection('projects/wri-datalab/cities/urban_land_use/V1')
ULUv2 = ee.ImageCollection('projects/wri-datalab/urban_land_use/V2')
ULU = ULU200.merge(ULU4000).merge(ULUv2)
ULU = ULU.select('lulc').reduce(ee.Reducer.firstNonNull()).rename('lulc') #//.clip(cityArea)//.updateMask(ULUMexico)
informal = ULU.mask(ULU.mask().gt(0)).remap([2, 3], [1, 1], 0)

In [10]:
# Get WorldPop data

pop = ee.ImageCollection('WorldPop/GP/100m/pop').filter(ee.Filter.equals('year', POP_YEAR))

In [21]:
# get vuln
f70_plus = ee.Image('projects/ee-tedwongwri/assets/mex_f_70plus_2020').rename("population")


In [11]:
def get_bbox(ee_obj):
    ee_geom = ee_obj.geometry()
    coords = ee_geom.bounds().getInfo()['coordinates'][0]
    left = coords[0][0]
    bottom = coords[0][1]
    right = coords[1][0]
    top = coords[2][1]
    return ({'N': top, 'S': bottom, 'E': right, 'W': left})

In [12]:
def geojson_to_ee(geojson):
    geo = json.loads(geojson)
    shapes = [Shape(f['geometry']) for f in geo['features']]
    shape_union = unary_union(shapes)

    return ee.FeatureCollection(ee.Geometry(json.loads(shapely.to_geojson(shape_union))))

In [24]:
def get_openspace(bbox, openspace_narrow=True):
    
    MIN_PATCHSIZE = [0, 10000][int(openspace_narrow)]

    query = '[out:json][bbox:{0}, {1}, {2}, {3}];('.format(bbox['S'], bbox['W'], bbox['N'], bbox['E'])
    query += 'way[leisure=park];>;'
    query += 'way[leisure=nature_reserve];>;'
    query += 'way[leisure=common];>;'
    query += 'way[leisure=playground];>;'
    query += 'way[leisure=pitch];>;'
    query += 'way[leisure=track];>;'
    if not openspace_narrow:
        query += 'way[boundary=protected_area];>;'
        query += 'way[boundary=national_park];>;'
    query += ');'
    query += 'out body;'

    api = overpy.Overpass()
    result = api.query(query)
    #oords = {i.id: [float(i.lon), float(i.lat)] for i in result.get_nodes()}

    #m = ee.Feature(ee.Geometry({'type': 'MultiPolygon', 'coordinates': [[[coords[node.id] for node in way.get_nodes(resolve_missing=True)] for way in result.ways if len(way.get_nodes(resolve_missing=True)) > 2]]}))
    #openspace_features = [ee.Feature(ee.Geometry({"type": "Polygon", "coordinates": [float(i.lon), float(i.lat)]} for i in j.get_nodes(resolve_missing=True))) for j in result.ways]
    #openspace_featureunion = ee.FeatureCollection(openspace_features).union()
    #m
    def utm_polygon(latlon_polygon):
        centroid_lon = latlon_polygon.centroid.xy[0][0]
        centroid_lat = latlon_polygon.centroid.xy[1][0]
        target_epsg = (32600 + [0, 100][int(centroid_lat < 0)]) + floor((180 + centroid_lon) / 6) + 1
        # EPSG is 32600 (or 32700 if lat is neg) + longitude zone. Each zone is six degrees, and first zone is 1.
        # Transform from EPSG:4326 to target EPSG
        project = pyproj.Transformer.from_crs(
            pyproj.CRS.from_epsg(4326), # source coordinate system
            pyproj.CRS.from_epsg(target_epsg), # destination coordinate system
            always_xy=True
        )
        return transform(project.transform, latlon_polygon)
    
    allpolys = [Polygon([[float(node.lon), float(node.lat)] for node in poly.nodes]) for poly in result.ways if len(poly.nodes) > 3]
    big_enough_polys = MultiPolygon([poly for poly in allpolys if utm_polygon(poly).area >= MIN_PATCHSIZE])
    big_enough_polys_geom = ee.Geometry(json.loads(shapely.to_geojson(big_enough_polys)))
    return ee.FeatureCollection(big_enough_polys_geom)

In [71]:
def mean_distance_to_openspace(country_code, geojson, informal_only=False):
    
    #boundary_id_aoi = '{0}-{1}'.format(boundary_georef.loc[i, 'geo_name'], boundary_georef.loc[i, 'aoi_boundary_name'])
    #boundary_path = '{0}/data/boundaries/boundary-{1}.geojson'.format(aws_s3_dir, boundary_id_aoi)
    #boundary_geo = requests.get(boundary_path).json()
    #boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    

    boundary_geo_ee = geojson_to_ee(geojson)
    bbox = get_bbox(boundary_geo_ee)
    
    #country_code = boundary_georef.loc[i, 'country_code']
    
    localpop = pop.filter(ee.Filter.equals('country', country_code)).select('population').first().clip(boundary_geo_ee)
    informal_pop = localpop.multiply(informal)
    
    pop_of_interest = [localpop, informal_pop][int(informal_only)]
    
    if pop_of_interest.mask().reduceRegion(reducer= ee.Reducer.sum(), geometry= boundary_geo_ee.geometry()).getInfo()['population'] == 0:
        return -9999

    # Get open space polygons
    #openspace_path = aws_s3_dir + '/data/open_space/openstreetmap/{0}-OSM-open_space-{1}.geojson'.format(boundary_id_aoi, OPENSPACE_YEAR)
    #openspace_geo = requests.get(openspace_path).json()
    #openspace_geo_ee = geemap.geojson_to_ee(openspace_geo)
    openspace_geo_ee = get_openspace(bbox)
    
    openspace_distance = openspace_geo_ee.distance(20000, 10)
    
    
    persondistancetoopenspace = (pop_of_interest.multiply(openspace_distance)).reduceRegion(reducer= ee.Reducer.sum(), geometry= boundary_geo_ee.geometry()).getInfo()['population']
    total_pop_of_interest = pop_of_interest.reduceRegion(reducer= ee.Reducer.sum(), geometry= boundary_geo_ee.geometry()).getInfo()['population']
    
    if total_pop_of_interest > 0:
        return persondistancetoopenspace / total_pop_of_interest
    else:
        return -9999

In [25]:
def fraction_near_openspace(country_code, geojson, openspace_narrow=True, use_vuln=False, informal_only=False):
    
    #boundary_id_aoi = '{0}-{1}'.format(boundary_georef.loc[i, 'geo_name'], boundary_georef.loc[i, 'aoi_boundary_name'])
    #boundary_path = '{0}/data/boundaries/boundary-{1}.geojson'.format(aws_s3_dir, boundary_id_aoi)
    #boundary_geo = requests.get(boundary_path).json()
    #boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    #bbox = get_bbox(boundary_geo_ee)
    
    
    #country_code = boundary_georef.loc[i, 'country_code']
    
    boundary_geo_ee = geojson_to_ee(geojson)
    bbox = get_bbox(boundary_geo_ee)
    
    bbox = get_bbox(boundary_geo_ee)
        
    if not use_vuln:
        localpop = pop.filter(ee.Filter.equals('country', country_code)).select('population').first().clip(boundary_geo_ee)
        informal_pop = localpop.multiply(informal)

        pop_of_interest = [localpop, informal_pop][int(informal_only)]
    else:
        vulnpop_country = ee.ImageCollection("WorldPop/GP/100m/pop_age_sex").filter(ee.Filter.eq('country', country_code)).filter(ee.Filter.eq('year', 2020)).first()
        pop_of_interest = vulnpop_country.select('M_70').rename("population").add(vulnpop_country.select('M_75')).add(vulnpop_country.select('M_80')).add(vulnpop_country.select('F_70')).add(vulnpop_country.select('F_75')).add(vulnpop_country.select('F_80'))
    
    #print(pop_of_interest.mask().reduceRegion(reducer= ee.Reducer.sum(), geometry= boundary_geo_ee.geometry()).getInfo())
    if pop_of_interest.mask().reduceRegion(reducer= ee.Reducer.sum(), geometry= boundary_geo_ee.geometry()).getInfo()['population'] == 0:
        return -9999

    openspace_geo_ee = get_openspace(bbox, openspace_narrow).union()
    
    buffer_geom = openspace_geo_ee.geometry().buffer(400, 10)
    
    nearpoptotal = pop_of_interest.reduceRegion(reducer=ee.Reducer.sum(), geometry=buffer_geom).getInfo()['population']
    total_pop = pop_of_interest.reduceRegion(reducer= ee.Reducer.sum(), geometry= boundary_geo_ee.geometry()).getInfo()['population']
    
    if total_pop > 0:
        return nearpoptotal / total_pop
    else:
        return -9999

In [15]:
from math import floor

In [37]:
results = {}
dones = []
for geo_info in geos:
    if (not geo_info[1] in dones) and geo_info[1] in ['Addis_Ababa', 'Bangalore', 'Bogota', 'Belem', 'Nairobi', 'Rio_de_Janerio', 'Beijing', 'Shenzhen', 'Mumbai', 'Guadalajara', 'Monterrey', 'Istanbul', 'Delhi', 'Jakarta']:
        dones.append(geo_info[1])
        print(geo_info[1])
        results['{0}-{1}-{2}'.format(geo_info[0], geo_info[1], geo_info[2])] = {
            #'mean_dist_to_openspace_informal': mean_distance_to_openspace(geo_info[0], geo_info[3], informal_only=True),
            #'mean_dist_to_openspace_all': mean_distance_to_openspace(geo_info[0], geo_info[3], informal_only=False),
            'popfraction_near_openspace_informal_narrowdef': fraction_near_openspace(geo_info[0], geo_info[3], use_vuln=False, informal_only=True),
            'popfraction_near_openspace_elderly_narrowdef': fraction_near_openspace(geo_info[0], geo_info[3], use_vuln=True, informal_only=False),
            'popfraction_near_openspace_all_narrowdef': fraction_near_openspace(geo_info[0], geo_info[3], informal_only=False),
            'popfraction_near_openspace_informal_expansivedef': fraction_near_openspace(geo_info[0], geo_info[3], openspace_narrow=False, use_vuln=False, informal_only=True),
            'popfraction_near_openspace_elderly_expansivedef': fraction_near_openspace(geo_info[0], geo_info[3], openspace_narrow=False, use_vuln=True, informal_only=False),
            'popfraction_near_openspace_all_expansivedef': fraction_near_openspace(geo_info[0], geo_info[3], openspace_narrow=False, use_vuln=False, informal_only=False)
        }
df = pd.DataFrame(results).transpose()
#df.to_csv('distance_to_openspace.csv')

Belem
Rio_de_Janerio
Beijing
Shenzhen
Bogota
Addis_Ababa
Bangalore
Delhi
Mumbai
Nairobi
Guadalajara
Monterrey
Istanbul
Jakarta


In [38]:
df

,popfraction_near_openspace_informal_narrowdef,popfraction_near_openspace_elderly_narrowdef,popfraction_near_openspace_all_narrowdef,popfraction_near_openspace_informal_expansivedef,popfraction_near_openspace_elderly_expansivedef,popfraction_near_openspace_all_expansivedef
BRA-Belem-ADM3,0.108086,0.151154,0.145556,0.338145,0.513958,0.493721
BRA-Rio_de_Janerio-ADM2,0.124715,0.219391,0.202086,0.715722,1.014428,0.860288
CHN-Beijing-area,0.236932,0.514565,0.488630,0.342083,0.713236,0.666046
CHN-Shenzhen-city,0.234339,0.341318,0.252449,0.322559,0.574974,0.434427
COL-Bogota-ADM2,0.420258,0.561252,0.504181,0.806177,0.963386,0.850730
ETH-Addis_Ababa-ADM4,0.017379,0.111807,0.097813,0.105544,0.313279,0.280821
IND-Bangalore-city,0.103585,0.215940,0.214035,0.449877,0.676736,0.666927
IND-Delhi-city,0.184616,0.432504,0.377483,0.360821,0.667420,0.544120
IND-Mumbai-city,0.171251,0.285408,0.267048,0.677202,0.760039,0.676523
KEN-Nairobi-ADM3,0.108409,0.230969,0.138118,0.339950,0.459926,0.353348


In [39]:
df.to_csv("openspace.csv")

In [68]:
df1.to_csv('distance_to_openspace_leisure_1ha.csv')

In [464]:
def export_openspace(i):
    boundary_id_aoi = '{0}-{1}'.format(boundary_georef.loc[i, 'geo_name'], boundary_georef.loc[i, 'aoi_boundary_name'])
    boundary_path = '{0}/data/boundaries/boundary-{1}.geojson'.format(aws_s3_dir, boundary_id_aoi)
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    geo_name = boundary_georef.iloc[i]['geo_name']
    
    bbox = get_bbox(boundary_geo_ee)
    
    openspace_feature = get_openspace(bbox)
    
    task = ee.batch.Export.table.toAsset(**{
      'collection': openspace_feature,
      'description': 'openspace_{0}'.format(geo_name),
      'assetId': 'users/tedwongwri/{0}'.format('openspace_{0}'.format(geo_name)),
    })
    task.start()
    

In [465]:
export_openspace(9)